In [3]:
# from sp500.scrape import scrape_sp500, create_sp500_df
import os

import boto3
import pandas as pd
import io
from datetime import datetime, timedelta
import pytz
from sqlalchemy import create_engine, text
from scrape import scrape_sp500, create_sp500_df
import mysql.connector
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
SP500_URL = "https://www.slickcharts.com/sp500"

In [7]:


with mysql.connector.connect(host=os.getenv('HOST'),
                                         database=os.getenv('DATABASE'),
                                         user=os.getenv('USERNAME'),
                                         password=os.getenv('PASSWORD')) as connection:
    print(connection)

In [12]:
def upload_df_to_sql(df, table):
    conn_string = f"mysql+mysqlconnector://{os.getenv('USERNAME')}:{os.getenv('PASSWORD')}@{os.getenv('HOST')}/{os.getenv('DATABASE')}"
    engine = create_engine(conn_string)
    with engine.connect() as connection:
        utc_now = pytz.utc.localize(datetime.utcnow())
        est_yesterday = utc_now.astimezone(
            pytz.timezone("America/New_York")
        ) - timedelta(1)

        with connection.begin():
            query = text("select max(Date) from Stocks")
            res = connection.execute(query)
            for line in res:
                if line[0] is None:
                    continue
                elif est_yesterday.date() == line[0]:
                    return

        date_col = [est_yesterday.date()] * df.shape[0]
        df["Date"] = date_col
        df = df[["Date", "Company", "Symbol", "Weight", "Price"]].copy()
        df["Date"] = df["Date"].astype("datetime64[ns]")

        df.to_sql(table, con=connection, if_exists="append", index=False)


soup_obj = scrape_sp500(SP500_URL)
df = create_sp500_df(soup_obj)

upload_df_to_sql(df, "Stocks")

In [98]:
SP500_URL = "https://www.slickcharts.com/sp500"

In [104]:
soup_obj = scrape_sp500(SP500_URL)
df = create_sp500_df(soup_obj)

In [121]:
client = boto3.client('s3')
response = client.list_objects_v2(Bucket="gurrastav")
def list_keys(bucket):
    client = boto3.client('s3')
    response = client.list_objects_v2(Bucket="gurrastav")
    if response.get("ResponseMetadata", {}).get("HTTPStatusCode") == 200:
        return [obj.get("Key") for obj in response.get("Contents")]

In [122]:
def upload_df_to_s3(df, bucket, dest_key):
    client = boto3.client('s3')
    csv_buffer = io.BytesIO()
    df.to_csv(csv_buffer)
    client.upload_fileobj(csv_buffer, bucket, dest_key)

csv_buffer = io.BytesIO()
df.to_csv(csv_buffer)

client.upload_fileobj(csv_buffer, "gurrastav", f"sp500/{datetime.datetime.now().strftime('%Y_%m_%d')}.csv")

In [125]:
for key in list_keys("gurrastav"):
    print(key)

sp500/
sp500/2023_06_10
sp500/2023_06_10.csv
sp500/testing


2023_06_10
